In [1]:
import pandas as pd
import numpy as np

In [2]:
from surprise import Dataset
from surprise import Reader

In [3]:
from UBCFBase import UBCFBase

In [4]:
reader = Reader(line_format='user item rating', sep=':')

In [5]:
data = Dataset.load_from_file('new_usr_ratings.txt', reader=reader)

In [6]:
data.split(n_folds=3)

In [7]:
trainsets = []
for trainset, testset in data.folds():
    trainsets.append(trainset)

In [8]:
first_trainset = trainsets[0]

In [9]:
first_trainset.n_users

970

In [10]:
algo = UBCFBase('new_ratings_all.txt')

In [11]:
algo.train(first_trainset)

C:\Users\rico\Miniconda2\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the behavior similarity matrix...
Done computing behavior similarity matrix


In [12]:
from sklearn.metrics.pairwise import pairwise_distances

In [13]:
temp = algo.trainset_assist_rating_df.groupby(['uid', 'comment_type']).size().unstack().fillna(0)

In [14]:
algo.UCI

uid
0      0.117279
1      0.090301
2      0.127673
3      0.163442
4      0.153344
5      0.101027
6      0.077351
7      0.191309
8      0.053859
9      0.055076
10     0.174408
11     0.048195
12     0.121666
13     0.075698
14     0.034325
15     0.492575
16     0.071068
17     0.040907
18     0.037947
19     0.050069
20     0.234244
21     0.028679
22     0.072327
23     0.077779
24     0.051670
25     0.154584
26     0.019319
27     0.039005
28     0.075119
29     0.299539
         ...   
940    0.147026
941    0.105521
942    0.055242
943    0.193244
944    0.069340
945    0.063571
946    0.089552
947    0.066948
948    0.038586
949    0.011173
950    0.010086
951    0.036092
952    0.000236
953    0.044640
954    0.041752
955    0.106726
956    0.003652
957    0.112007
958    0.001767
959    0.128055
960    0.067291
961    0.086438
962    0.111168
963    0.143347
964    0.043632
965    0.089578
966    0.078538
967    0.098069
968    0.050907
969    0.000059
dtype: float64

In [15]:
def f(df):
    df['hot_w'] = 1 / np.log(math.e + df['hot'] - 1)
    temp = df.groupby('comment_type')['hot_w'].sum()
    
    if 1 in temp.index:
        return temp[1] / len(temp)
    else:
        return 0

In [17]:
algo.behavior_sim

array([[  0.00000000e+00,   7.10008890e-01,   9.98260390e-01, ...,
          2.30045779e+00,   2.63454709e+00,   2.63631820e+00],
       [  7.10008890e-01,   0.00000000e+00,   2.91118884e-01, ...,
          1.59090037e+00,   1.92461795e+00,   1.92632966e+00],
       [  9.98260390e-01,   2.91118884e-01,   0.00000000e+00, ...,
          1.30250779e+00,   1.63730525e+00,   1.64047580e+00],
       ..., 
       [  2.30045779e+00,   1.59090037e+00,   1.30250779e+00, ...,
          5.26835606e-09,   3.36653183e-01,   3.47443649e-01],
       [  2.63454709e+00,   1.92461795e+00,   1.63730525e+00, ...,
          3.36653183e-01,   0.00000000e+00,   5.08480918e-02],
       [  2.63631820e+00,   1.92632966e+00,   1.64047580e+00, ...,
          3.47443649e-01,   5.08480918e-02,   0.00000000e+00]])